In [1]:
import os 
import sys
import pandas as pd
import numpy as np

directory = os.getcwd()

#cwd = directory.replace('Strat_7', '\module_1')
cwd1 = directory.replace('Strat_7', '')
#os.chdir(cwd)
#sys.path.append(cwd)
sys.path.append(cwd1)
import Preprocessing_functions as pf 
import calculateMaxDD 
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib
#os.getcwd()

In [2]:
ticker = 'XLV'
time_period = '120mo'

df = pf.downlaod_symbol_data(ticker, period= time_period)
try:
    df = df.drop(columns=['Stock Splits', 'Dividends', 'Capital Gains'])
except KeyError:
    print("Columns not available (see above line of code)")
    
df = pf.create_momentum_feat(df, symbol=ticker) ### need to inspect in more detail how the create momemntum features work and the shift in this case
df = pf.technical_indicators(df,MA_DIVERGENCE=True)
df = pf.format_idx_date(df)

#df['prev_close'] = df['Close'].shift(1)
#df['tom_open'] = df['Open'].shift(-1)

df['overnight_pct'] = (df['Open'].shift(-1) - df['Close']) / df['Close']
#df['overnight_pct'] = (df['Open'] - df['Close'].shift(1)) / df['Close'].shift(1) WRONG - OVERFITTING!

df = pf.add_market_feature('^VIX', time_period=time_period, data = df, price=True)

df = df[df.index <= '2024-02-01']

df = df.dropna()

df.tail()

,Open,High,Low,Close,Volume,open_low,open_close,open_high,high_low,low_close,...,diff_Close_8,diff_Close_25,diff_Close_50,diff_Close_100,diff_Close_200,diff_8_50,diff_50_100,diff_50_200,overnight_pct,^VIX_level
Date,,,,,,,,,,,,,,,,,,,,,
2024-01-26,136.73,136.85,136.22,136.56,5759100,0.372998,0.124333,-0.087764,0.460358,-0.249596,...,-0.07250,0.8860,4.5428,7.5483,7.78660,4.61530,3.0055,3.24380,0.000952,13.26
2024-01-29,136.69,137.54,136.60,137.53,6229600,0.065842,-0.614529,-0.621845,0.683438,-0.680820,...,0.79250,1.5608,5.2586,8.4282,8.71730,4.46610,3.1696,3.45870,0.001600,13.60
2024-01-30,137.75,138.08,137.09,137.84,5940300,0.479129,-0.065336,-0.239564,0.716976,-0.547086,...,0.95625,1.6268,5.3102,8.6380,8.99485,4.35395,3.3278,3.68465,0.007618,13.31
2024-01-31,138.89,139.00,137.62,137.65,10223100,0.914393,0.892793,-0.079199,0.992806,-0.021799,...,0.65375,1.2268,4.8762,8.3554,8.76805,4.22245,3.4792,3.89185,0.001235,14.35
2024-02-01,137.82,139.44,137.15,139.44,8503600,0.486141,-1.175446,-1.175446,1.642283,-1.669705,...,2.18250,2.7468,6.3814,10.0349,10.51180,4.19890,3.6535,4.13040,-0.000359,13.88


In [3]:
print('Start date of dataframe: ', df.index.min())
print('End date of dataframe: ', df.index.max())

Start date of dataframe:  2016-03-09 00:00:00
End date of dataframe:  2024-02-01 00:00:00


In [4]:
y = df.pop('overnight_pct')
X = df.copy()
X = X.drop(columns= ['Open', 'High', 'Low', 'Close',])

# Split data into train (60%), validation (20%), and test (20%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [5]:
X.columns

Index(['Volume', 'open_low', 'open_close', 'open_high', 'high_low',
       'low_close', 'high_close', 'gap', 'XLV_mom1', 'XLV_mom2', 'XLV_mom3',
       'XLV_mom4', 'XLV_mom5', 'XLV_mom10', 'XLV_mom15', 'XLV_mom20',
       'XLV_mom60', 'XLV_mom120', 'XLV_mom180', 'XLV_mom240', 'ATR', 'MACD',
       'diff_Close_8', 'diff_Close_25', 'diff_Close_50', 'diff_Close_100',
       'diff_Close_200', 'diff_8_50', 'diff_50_100', 'diff_50_200',
       '^VIX_level'],
      dtype='object')

In [6]:
model_columns = pd.DataFrame(X.columns, columns=['features'])

model_columns.to_csv(directory + f'/model_features/{ticker}_features_{len(X.columns)}.csv', index = False)


In [7]:
# Define Random Forest model
rf = RandomForestRegressor(random_state=42)

# Grid Search for best hyperparameters
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

grid_search = GridSearchCV(rf, param_grid, cv=10, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Best parameters from Grid Search
best_rf = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

Fitting 10 folds for each of 81 candidates, totalling 810 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}


In [8]:
# Cross-validation with 10-folds
cv_scores = cross_val_score(best_rf, X_train, y_train, cv=10, scoring='neg_mean_squared_error')
rmse_cv = np.sqrt(-cv_scores)
print(f"10-Fold CV RMSE: {rmse_cv.mean():.4f} ± {rmse_cv.std():.4f}")

10-Fold CV RMSE: 0.0061 ± 0.0012


In [9]:
# Evaluate on validation set
y_val_pred = best_rf.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"Validation RMSE: {rmse_val:.4f}")

Validation RMSE: 0.0066


In [10]:
X_val.head()

,Volume,open_low,open_close,open_high,high_low,low_close,high_close,gap,XLV_mom1,XLV_mom2,...,MACD,diff_Close_8,diff_Close_25,diff_Close_50,diff_Close_100,diff_Close_200,diff_8_50,diff_50_100,diff_50_200,^VIX_level
Date,,,,,,,,,,,,,,,,,,,,,
2021-08-24,6548100,0.638779,0.358339,-0.007790,0.646518,-0.282242,-0.366101,0.000000,-0.36,-0.32,...,0.021613,0.19750,2.3604,5.4056,9.3348,15.41605,5.20810,3.9292,10.01045,17.22
2021-01-13,5259600,0.366569,-0.320748,-0.668988,1.028675,-0.689845,-0.345926,-0.054955,0.27,-0.82,...,0.360671,0.93125,3.6136,5.3352,8.1899,12.11135,4.40395,2.8547,6.77615,22.21
2022-07-19,6585700,0.430090,-0.632963,-0.803376,1.223635,-1.067645,-0.169055,1.066186,1.71,-0.44,...,0.077453,0.62000,2.7424,1.6474,-1.1678,-1.35720,1.02740,-2.8152,-3.00460,24.50
2018-11-02,12953000,2.217295,1.576743,-0.172456,2.385637,-0.655077,-1.746188,0.957592,-0.63,0.63,...,-0.099559,0.77375,-1.9396,-2.6334,-0.3619,2.64900,-3.40715,2.2715,5.28240,19.51
2017-07-06,6367700,1.002147,0.858984,-0.186113,1.186053,-0.144613,-1.043155,-0.427655,-1.28,-0.70,...,-0.121095,-0.67875,0.3620,1.7654,2.9355,5.64290,2.44415,1.1701,3.87750,12.54


In [11]:
# Final evaluation on test set
y_test_pred = best_rf.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f"Test RMSE: {rmse_test:.4f}")

Test RMSE: 0.0055


In [12]:
os.chdir(cwd.replace('module_1', 'Strat_7\models'))

NameError: name 'cwd' is not defined

In [ ]:
os.getcwd() + '\models'

In [ ]:
# Save the best model
os.chdir(os.getcwd() + '\models')
joblib.dump(best_rf, f"{ticker}_overnight_regression_random_forest.pkl")
print(f"Model saved as {ticker}_overnight_regression_random_forest.pkl")



# Save the best model
joblib.dump(best_rf, "best_random_forest.pkl")
print("Model saved as best_random_forest.pkl")

# Load the model for future use
loaded_model = joblib.load("best_random_forest.pkl")
y_loaded_pred = loaded_model.predict(X_test)

#### BACKTESTING

In [ ]:
#ticker = 'SPY'

df1 = pf.downlaod_symbol_data(ticker, period= '360mo')
try:
    df1 = df1.drop(columns=['Stock Splits', 'Dividends', 'Capital Gains'])
except KeyError:
    print("Columns not available (see above line of code)")
    
df1 = pf.create_momentum_feat(df1, symbol=ticker) ### need to inspect in more detail how the create momemntum features work and the shift in this case
df1 = pf.technical_indicators(df1,MA_DIVERGENCE=True)
df1 = pf.format_idx_date(df1)
df1 = pf.add_market_feature('^VIX', time_period='360mo', data = df1, price=True)

df1 = df1.dropna()

model_start_date = df.index.min()

df1 = df1[df1.index < model_start_date]

df1.head()

In [ ]:
X.columns

In [ ]:
df2 = df1[X.columns]

df2.head()

In [ ]:
#os.chdir(cwd.replace('module_1', 'Strat_7\models'))
os.getcwd()

In [ ]:
best_rf = joblib.load(f"{ticker}_overnight_regression_random_forest.pkl")

In [ ]:
df1['predictions'] = best_rf.predict(df2)
df1['overnight_pct'] = (df1['Open'].shift(-1) - df1['Close']) / df1['Close']
df1['action'] = np.where(df1['predictions'] > 0, 'BUY', 'SELL')
df1.head()

In [ ]:
df1 = df1.dropna()
df1['prev_close'] = df1['Close'].shift()
df1['tom_open'] = df1['Open'].shift(-1)

cols = df1.columns
cols = [i for i in cols if i not in X.columns]

df1 = df1[cols]

df1.head()

In [ ]:
import matplotlib.pyplot as plt 

df1['error'] = df1['predictions'] - df1['overnight_pct']

plt.figure(figsize=[10,7])
plt.plot(df1.index, df1['error'] , color = 'b')
plt.title('Overning Error in PCT points')
plt.xlabel('Date')
plt.ylabel('Pct Difference')

In [ ]:
df1.head()

In [ ]:
df1['return'] = np.where(df1['action'] == 'SELL', df1['overnight_pct'] * (-1), df1['overnight_pct'])
df1.head()

In [ ]:
df1['cum_ret'] = df1['return'].cumsum()

plt.figure(figsize=[10,7])
plt.plot(df1.index, df1['cum_ret'] , color = 'b')
plt.title('Backtest Cummulative Return PCT')
plt.xlabel('Date')
plt.ylabel('Cummulative Return PCT')

In [ ]:
##### LOSSES
df1[df1['return'] < 0]

In [ ]:

#####   MAX DRAWDOWN
#from calculateMaxDD import calculateMaxDD

cum_ret = np.cumprod(1+ df1['return']) - 1
maxDrawdown, maxDrawdownDuration, startDrawdownDay=calculateMaxDD.calculateMaxDD(cum_ret.values)

#####   SHARPE RATIO
sharpe_ratio = round(np.sqrt(252) * np.mean(df1['return']) / np.std(df1['return']),2)

#####   AVG YEARLY RETURN AND STD
mean_ret = df1['return'].mean() * 252
std = df1['return'].std()*np.sqrt(252)

print(f'Sharpe Ratio: {sharpe_ratio}')
print(f'Maximum Drawdown: {round(maxDrawdown,4)}')
print(f'Max Drawdown Duration: {maxDrawdownDuration} days' )
print(f'Start day Drawdown: {startDrawdownDay}')
print(f"Average Yearly Return: {round(mean_ret*100, 2)} %")

In [ ]:
rets = df1[['return']]
rets.to_csv(directory + '\strat_returns' + f'\{ticker}.csv')

In [ ]:
directory

In [ ]:
df1.predictions.describe()